In [7]:
pip install tensorflow opencv-python numpy matplotlib tqdm



[notice] A new release of pip is available: 24.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm


In [ ]:
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 20



train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    r'C:\Users\Subhodeep\Desktop\Deepfake\dataset\train',
    target_size=(128, 128),
    batch_size=64,
    class_mode='binary'
)

val_data = val_datagen.flow_from_directory(
    r'C:\Users\Subhodeep\Desktop\Deepfake\dataset\vaild',  
    target_size=(128, 128),
    batch_size=64,
    class_mode='binary'
)




Found 2041 images belonging to 2 classes.
Found 2041 images belonging to 2 classes.


In [ ]:
def build_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn_model()
cnn_model.summary()


c:\Users\Subhodeep\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,517,185 (24.86 MB)

 Trainable params: 6,516,737 (24.86 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:

cnn_model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

cnn_model.save("deepfake_cnn.h5")


Epoch 1/20


c:\Users\Subhodeep\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 180s 5s/step - accuracy: 0.5483 - loss: 3.3471 - val_accuracy: 0.5076 - val_loss: 1.0975
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 126s 4s/step - accuracy: 0.5831 - loss: 0.7444 - val_accuracy: 0.5296 - val_loss: 3.1224
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - accuracy: 0.6416 - loss: 0.6553 - val_accuracy: 0.5296 - val_loss: 3.8731
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 123s 4s/step - accuracy: 0.6421 - loss: 0.6327 - val_accuracy: 0.5296 - val_loss: 3.7666
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 120s 3s/step - accuracy: 0.6935 - loss: 0.5682 - val_accuracy: 0.5296 - val_loss: 2.4534
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 123s 4s/step - accuracy: 0.6972 - loss: 0.5706 - val_accuracy: 0.5311 - val_loss: 1.3533
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 118s 4s/step - accuracy: 0.7234 - loss: 0.5314 - val_accuracy: 0.5326 - val_loss: 1.3234
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 113s 3s/step - accuracy: 0.7405 - loss: 0.5099 - val_accuracy: 0.5439 - val_loss: 1.

In [ ]:
def predict_image(model, image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)[0][0]
    return "Fake" if prediction > 0.5 else "Real"


print(predict_image(cnn_model, "Convex_lens_(magnifying_glass)_and_upside-down_image.jpg"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
Real


In [ ]:
def predict_video(model, video_path, frame_interval=5):
    cap = cv2.VideoCapture(video_path)
    fake_frames = 0
    total_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if total_frames % frame_interval == 0:
            frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame_resized = frame_resized / 255.0
            frame_resized = np.expand_dims(frame_resized, axis=0)

            prediction = model.predict(frame_resized)[0][0]
            if prediction > 0.5:
                fake_frames += 1

        total_frames += 1

    cap.release()
    return "Fake Video" if fake_frames > total_frames // 2 else "Real Video"

print(predict_video(cnn_model, "6266253-uhd_2160_3840_25fps.mp4"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━